<a href="https://colab.research.google.com/github/Kavitakg/My-First-Repository/blob/main/attrition_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [39]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [40]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [41]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development
...,...,...
1465,No,Research & Development
1466,No,Research & Development
1467,No,Research & Development
1468,No,Sales


In [42]:
# Look at value counts to decide which encoder to use
y_df['Attrition'].value_counts(), y_df['Department'].value_counts()

(Attrition
 No     1233
 Yes     237
 Name: count, dtype: int64,
 Department
 Research & Development    961
 Sales                     446
 Human Resources            63
 Name: count, dtype: int64)

In [43]:
# Create a list of at least 10 column names to use as X data
column_names = ['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'HourlyRate', 'JobRole',
                'NumCompaniesWorked', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion']


# Create X_df using your selected columns
X_df = attrition_df[column_names]
X_df.head()

# Show the data types for X_df
X_df.dtypes


,0
Age,int64
DistanceFromHome,int64
Education,int64
EnvironmentSatisfaction,int64
HourlyRate,int64
JobRole,object
NumCompaniesWorked,int64
YearsAtCompany,int64
YearsInCurrentRole,int64
YearsSinceLastPromotion,int64


In [44]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)


In [45]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
# --- Convert categorical features in X to numerical using OneHotEncoder ---
categorical_features_X = X_train.select_dtypes(include=['object']).columns.tolist()
encoder_X = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_train_encoded = encoder_X.fit_transform(X_train[categorical_features_X])
X_test_encoded = encoder_X.transform(X_test[categorical_features_X])

X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoder_X.get_feature_names_out(categorical_features_X), index=X_train.index)
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoder_X.get_feature_names_out(categorical_features_X), index=X_test.index)

X_train = pd.concat([X_train.drop(categorical_features_X, axis=1), X_train_encoded_df], axis=1)
X_test = pd.concat([X_test.drop(categorical_features_X, axis=1), X_test_encoded_df], axis=1)


In [46]:
X_df['Education'].value_counts()

,count
Education,
3,572
4,398
2,282
1,170
5,48


In [47]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [48]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
# --- Convert categorical features in y to numerical using OneHotEncoder ---
categorical_features_y = y_train.select_dtypes(include=['object']).columns.tolist()
encoder_y = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
y_train_encoded = encoder_y.fit_transform(y_train[categorical_features_y])
y_test_encoded = encoder_y.transform(y_test[categorical_features_y])

# Initialize attrition_encoded and department_encoded
department_encoded = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
department_encoded.fit_transform(y_df[['Department']])
encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department_encoded = encoder.transform(y_train[['Department']])
y_test_department_encoded = encoder.transform(y_test[['Department']])


In [49]:
# Create a OneHotEncoder for the Attrition column
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Initialize attrition_encoded and department_encoded
attrition_encoded = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
attrition_encoded.fit_transform(y_df[['Attrition']])
encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = encoder.transform(y_train[['Attrition']])
y_test_attrition_encoded = encoder.transform(y_test[['Attrition']])


## Create, Compile, and Train the Model

In [50]:
# Find the number of columns in the X training data
input_dim = X_train_scaled.shape[1]
print(input_dim)

# Create the input layer
input_layer = layers.Input(shape=(input_dim,))

# Create at least two shared layers
shared_layer1 = layers.Dense(units=64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(units=32, activation='relu')(shared_layer1)

18


In [51]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
departement_hidden_layer = layers.Dense(units=12, activation='relu')(shared_layer2)

# Create the output layer
department_output_layer = layers.Dense(y_train_department_encoded.shape[1],
                                       activation='softmax',
                                       name='department_output')(departement_hidden_layer)


In [52]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(units=12, activation='relu')(shared_layer2)

# Create the output layer
attrition_output_layer = layers.Dense(y_train_attrition_encoded.shape[1],
                                       activation='softmax',
                                       name='attrition_output')(attrition_hidden_layer)


In [53]:
# Create the model
model = Model(inputs=input_layer,
              outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'categorical_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 18)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 64)             │          1,216 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 32)             │          2,080 │ dense_8[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 12)             │            396 │ dense_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 12)             │            396 │ dense_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             39 │ dense_10[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             26 │ dense_11[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,153 (16.22 KB)

 Trainable params: 4,153 (16.22 KB)

 Non-trainable params: 0 (0.00 B)

In [54]:
# Train the model
result = model.fit(X_train_scaled,
                   {'department_output': y_train_department_encoded, 'attrition_output': y_train_attrition_encoded},
                   epochs=100,
                   batch_size=32,
                   validation_data=(X_test_scaled, {'department_output': y_test_department_encoded,
                                                    'attrition_output': y_test_attrition_encoded}), verbose=1)


Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - attrition_output_accuracy: 0.8187 - department_output_accuracy: 0.2663 - loss: 1.6349 - val_attrition_output_accuracy: 0.8673 - val_department_output_accuracy: 0.9116 - val_loss: 1.2098
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8234 - department_output_accuracy: 0.9043 - loss: 1.1625 - val_attrition_output_accuracy: 0.8673 - val_department_output_accuracy: 0.9286 - val_loss: 0.7264
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8206 - department_output_accuracy: 0.9407 - loss: 0.6995 - val_attrition_output_accuracy: 0.8673 - val_department_output_accuracy: 0.9422 - val_loss: 0.5263
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8320 - department_output_accuracy: 0.9654 - loss: 0.5347 - val_attrition_output_accuracy: 0.8673 - val_department_output_accuracy: 0.9592 - val_loss: 0.4727
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3

In [55]:
# Evaluate the model with the testing data
evaluation = model.evaluate(x=X_test_scaled,
 y={'department_output': y_test_department_encoded,
    'attrition_output': y_test_attrition_encoded}, verbose=1)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8265 - department_output_accuracy: 0.9470 - loss: 1.1937 


In [56]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {evaluation[2]}")
print(f"Attrition Accuracy: {evaluation[4]}")

Department Accuracy: 0.9489796161651611


IndexError: list index out of range

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is not the best metric to use on this data as it does not account for class imbalances. The no. of employees leaving the company or attrition  is lesser than employees that stay, and accuracy can be inaccurate. We have to use alternative metrics such as balanced accuracy, precision, recall, or F1 score that can provide more insight into the model performance on each class, in particular; the minority class. These metrics take into consideration the true positivie, false positives and false negative rates, indicating more comprehensive view of the model's effectiveness.

2. The choice of activation functions for output layers typically depends on the type of problem being solved:
For multi-class classification problems, the softmax activation function is preferred. Softmax outputs a probability distribution across multiple classes, ensuring that the sum of the probabilities equals 1. This makes it suitable for problems where each instance belongs to one of several classes. In this model, we have department_output and attrition_output. The softmax activation function allows for interpreting the outputs as the likelihood of each class, and enables better decision-making based off the predicted probabilities.

3. Feature Engineering: Create new features that may provide additional insights, such as tenure with the company, performance ratings, or engagement scores. Features that capture employee interactions, such as participation in training programs or team projects, could also be valuable.

Data Collection: Gather more comprehensive data, including employee feedback from surveys, exit interviews, and historical data on promotions and transfers. This can help identify patterns related to attrition and departmental fit.

Handling Missing Data: Implement strategies for dealing with missing data, such as imputation techniques or using models that can handle missing values directly.

Hyperparameter Tuning: Use techniques like grid search or random search to optimize the hyperparameters of the model, which can lead to better performance.

Model Selection: Experiment with different algorithms, such as decision trees, random forests, gradient boosting machines, or neural networks, to find the best fit for the data.

Cross-Validation: Implement k-fold cross-validation to ensure that the model's performance is consistent across different subsets of the data and to avoid overfitting.

Class Imbalance Techniques: If the dataset is imbalanced (e.g., significantly more employees staying than leaving), consider using techniques like oversampling the minority class, undersampling the majority class, or applying algorithms that are robust to class imbalance.

Regularization: Apply regularization techniques (like L1 or L2 regularization) to reduce overfitting, especially if the model has a large number of features.

Ensemble Methods: Combine predictions from multiple models (e.g., using bagging or boosting techniques) to improve overall predictive performance.

Feedback Loop: Establish a feedback mechanism to continuously update the model with new data and insights as employee behaviors and organizational dynamics change over time.

By implementing these strategies, the HR model can become more accurate and effective in predicting employee attrition and identifying the best-fit department for each employee.